# Perturbation Confusion

This notebook demonstrates the "perturbation confusion" issue referenced in Section 1.1.1 of the memo paper, and shows how memo addresses it.

The problem set-up is that Alice has to choose between an indoor and outdoor restaurant, and has a slight preference for outdoor over indoor (utility 11 vs 10). However, there is a 50% chance of snow, and if it snows she greatly prefers indoor seating over outdoor (cost of 0 vs 100). We thus expect that Alice will "play it safe" and pick indoor seating.

However, in WebPPL, a naïve implementation of this problem leads Alice to pick outdoor seating. This is because this model erroneously gives Alice "control" over the weather.

In [1]:
%%bash
webppl <(cat <<EOF

Infer(function() {
    var r = uniformDraw(["outdoor", "indoor"]);
    var s = uniformDraw(["sunny", "stormy"]);
    factor(
        (r === "outdoor" ? 11 : 10) +
        (s === "stormy" && r === "outdoor" ? -100 : 0)
    );
    return r;
}).getDist();

EOF)

{
  '"indoor"': { val: 'indoor', prob: 0.4238831152341708 },
  '"

outdoor"': { val: 'outdoor', prob: 0.576116884765829 }
}


Let's implement this same model in memo. First, some imports and boilerplate.

In [2]:
from memo import memo, domain
from enum import IntEnum
import jax
import jax.numpy as np

class Restaurant(IntEnum): Indoor = 0; Outdoor = 1
class Weather(IntEnum): Sunny = 0; Stormy = 1
@jax.jit
def utility(r, s):
    return (np.where(r == Restaurant.Outdoor, 11, 10) +
            np.where((s == Weather.Stormy) & (r == Restaurant.Outdoor), -100, 0))

Now, we can write our memo model. Notice that _alice_ chooses the restaurant ($r$) but the _world_ chooses whether or not it snows ($s$). memo correctly predicts that Alice has an overwhelming preference for indoor seating.

In [3]:
@memo
def model[r: Restaurant]():
    alice: chooses(r in Restaurant, wpp=imagine[
        world: chooses(s in Weather, wpp=0.5),
        exp(E[utility(r, world.s)])
    ])
    return Pr[alice.r == r]

for r in Restaurant:
    print(f'P({r.name}) = {model()[r]}')

P(Indoor) = 1.0
P(Outdoor) = 5.242885696424093e-22


If for some reason we really _did_ explicitly mean for Alice to be able to choose both $r$ and $s$ (as implied by the erroneous WebPPL model), we can do that in memo as well. We intentionally have Alice choose jointly from tuples $(r, s)$. memo has built-in utilities for working with such tuples.

In [4]:
RS = domain(r=len(Restaurant), s=len(Weather))
get_r, get_s = jax.jit(RS.r), jax.jit(RS.s)

@memo
def model_[r: Restaurant]():
    alice: chooses(rs in RS, wpp=exp(utility(get_r(rs), get_s(rs))))
    return Pr[get_r(alice.rs) == r]

for r in Restaurant:
    print(f'P({r.name}) = {model_()[r]}')

P(Indoor) = 0.4238831102848053
P(Outdoor) = 0.5761168599128723


Notice that this gives us exactly the same output as the WebPPL version with perturbation confusion.